In [1]:
from src.FunctionsProperties import *

## Create all_files

In [2]:
# Create folders to Results
make_results_folders()
move_to_data()

In [3]:
# ==> READ DOCUMENTATION OF FUNCTION all_properties_file(N,dim, alpha_a, alpha_g) BEFORE RUN THAT CELL!! <==
N = [5000, 10000, 20000, 40000, 80000, 100000, 160000, 320000, 640000]
#N = [5000]
dim = [1, 2, 3, 4]

for n in N:
    for d in dim:
        all_combinations_ag =  extract_alpha_values(n, d)
        for i in range(len(all_combinations_ag)):
            try:
                # READ DOCUMENTATION BEFORE RUN THAT CELL
                all_properties_file(n, d, all_combinations_ag[i][0], all_combinations_ag[i][1])
                fixing_data(n, d, all_combinations_ag[i][0], all_combinations_ag[i][1])
            except FileNotFoundError:
                pass
            #remove_cod_file_column(n, d, all_combinations_ag[i][0], all_combinations_ag[i][1])
clear_output()

In [4]:
N = [5000, 10000, 20000, 40000, 80000, 100000 ,160000, 320000]
dim = [1, 2, 3, 4]
format_file(N, dim)
all_data(N, dim)

5000 1 8.0 3.0
5000 1 4.0 4.0
5000 1 1.0 9.0
5000 1 2.0 14.0
5000 1 0.0 2.0
5000 1 9.0 3.0
5000 1 36.0 2.0
5000 1 16.0 2.0
5000 1 6.0 8.0
5000 1 5.0 0.0
5000 1 8.0 7.0
5000 1 6.0 6.0
5000 1 0.0 3.0
5000 1 4.0 7.0
5000 1 1.0 4.0
5000 1 7.0 8.0
5000 1 1.0 2.0
5000 1 11.0 2.0
5000 1 2.0 0.1
5000 1 7.0 4.0
5000 1 4.0 8.0
5000 1 6.0 2.0
5000 1 9.0 7.0
5000 1 4.0 9.0
5000 1 2.0 3.0
5000 1 1.0 8.0
5000 1 8.0 5.0
5000 1 5.0 2.0
5000 1 1.5 2.0
5000 1 8.0 2.0
5000 1 14.0 2.0
5000 1 9.0 9.0
5000 1 8.0 0.0
5000 1 8.0 0.1
5000 1 4.0 3.0
5000 1 3.0 1.0
5000 1 8.0 4.0
5000 1 7.0 9.0
5000 1 9.0 0.1
5000 1 21.0 2.0
5000 1 8.0 1.0
5000 1 4.0 2.0
5000 1 8.0 8.0
5000 1 9.0 4.0
5000 1 0.0 5.0
5000 1 4.0 0.1
5000 1 5.0 0.1
5000 1 9.0 5.0
5000 1 7.0 0.0
5000 1 0.0 0.1
5000 1 6.5 2.0
5000 1 4.0 6.0
5000 1 8.0 6.0
5000 1 15.0 2.0
5000 1 7.5 2.0
5000 1 10.0 2.0
5000 1 2.0 4.0
5000 1 2.0 11.0
5000 1 1.0 1.0
5000 1 9.0 1.0
5000 1 9.0 2.0
5000 1 3.0 8.0
5000 1 2.0 6.0
5000 1 8.0 9.0
5000 1 2.0 7.0
5000 1 0.0 0.0
5

## Statistical Analysis

In [5]:
df = pd.read_csv("../../data/all_data.txt", delimiter=" ")
df = df[df["alpha_g"]>=1.0]
df.to_csv("../../data/all_data.txt", index=False, sep=' ')
df.head()

,N,dim,alpha_a,alpha_g,N_samples,short_mean,short_err,short_std,diameter_mean,diameter_err,diameter_std,ass_coeff_mean,ass_coeff_err,ass_coeff_std
0,5000,1,8.0,3.0,51,15.264582,0.141807,1.012706,36.254902,0.510166,3.643312,0.040476,0.001568,0.011196
1,5000,1,4.0,4.0,51,14.834792,0.175089,1.250386,35.686275,0.561853,4.012432,0.015934,0.001676,0.011971
2,5000,1,1.0,9.0,51,11.606915,0.150811,1.077005,28.274510,0.293830,2.098365,-0.095851,0.003681,0.026290
3,5000,1,2.0,14.0,9787,14.726629,0.016291,1.611699,34.694391,0.040366,3.993390,-0.039712,0.000165,0.016287
4,5000,1,0.0,2.0,1052,8.583539,0.015994,0.518760,22.595057,0.055319,1.794243,-0.075066,0.000479,0.015540


In [6]:
N = [5000, 10000, 20000, 40000, 80000, 160000, 320000]
alpha_filter = [0.0, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
dimensions = [1, 2, 3, 4]

f_df = df[
    (df['N'].isin(N)) &
    (df['alpha_a'].isin(alpha_filter)) &
    (df['dim'].isin(dimensions)) &
    (df['alpha_g'] == 2.0)
].sort_values(by=["alpha_a"]).reset_index(drop=True)
f_df

,N,dim,alpha_a,alpha_g,N_samples,short_mean,short_err,short_std,diameter_mean,diameter_err,diameter_std,ass_coeff_mean,ass_coeff_err,ass_coeff_std
0,5000,1,0.0,2.0,1052,8.583539,0.015994,0.518760,22.595057,0.055319,1.794243,-0.075066,0.000479,0.015540
1,5000,4,0.0,2.0,1052,8.593929,0.016083,0.521638,22.682510,0.056902,1.845600,-0.075923,0.000492,0.015973
2,160000,1,0.0,2.0,12,12.124283,0.154200,0.534164,34.250000,0.509382,1.764550,-0.024083,0.002092,0.007248
3,10000,1,0.0,2.0,3558,9.276311,0.008603,0.513134,24.766161,0.030128,1.797090,-0.060091,0.000223,0.013273
4,80000,4,0.0,2.0,131,11.380191,0.042103,0.481896,31.786260,0.146202,1.673355,-0.030689,0.000654,0.007482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,10000,2,15.0,2.0,51,15.841447,0.100122,0.715014,37.901961,0.423112,3.021621,0.027051,0.001207,0.008617
416,5000,3,15.0,2.0,101,13.550015,0.078821,0.792137,32.128713,0.267613,2.689473,0.007534,0.001351,0.013582
417,5000,4,15.0,2.0,101,13.078857,0.080510,0.809114,30.841584,0.266526,2.678554,0.003922,0.001355,0.013615
418,320000,3,15.0,2.0,7,21.316071,0.262507,0.694528,53.285714,0.993174,2.627691,0.019858,0.000693,0.001834


In [7]:
# Inicializar o dicionário vazio
data_percent = {"N": [], "dim": [], "alpha_a": [], "alpha_g": [], "per_short": [],"std_short":[] ,"per_ass": [],"std_ass":[] ,"per_dia": [], "std_dia":[]}

# Iterar pelas linhas do DataFrame
for _, row in f_df.iterrows():
    data_percent["N"].append(row["N"])
    data_percent["dim"].append(row["dim"])
    data_percent["alpha_a"].append(row["alpha_a"])
    data_percent["alpha_g"].append(row["alpha_g"])
    
    data_percent["std_short"].append(row["short_std"])
    data_percent["std_ass"].append(row["ass_coeff_std"])
    data_percent["std_dia"].append(row["diameter_std"])
    
    # Calcular os percentuais
    per_short = (row["short_err"] / row["short_mean"]) * 100 if row["short_mean"] != 0 else 0
    per_ass = (row["ass_coeff_err"] / row["ass_coeff_mean"]) * 100 if row["ass_coeff_mean"] != 0 else 0
    per_dia = (row["diameter_err"] / row["diameter_mean"]) * 100 if row["diameter_mean"] != 0 else 0
    
    # Adicionar os percentuais no dicionário
    data_percent["per_short"].append(per_short)
    data_percent["per_ass"].append(per_ass)
    data_percent["per_dia"].append(per_dia)

# Exibir o dicionário
df_per = pd.DataFrame(data=data_percent)
df_per.to_csv("percent_err.txt", index=False, sep=' ')

In [8]:
# Fix just assortativity value
filtered_df = df_per[
    (df_per['per_short'] >= 10) | 
    (df_per['per_ass'] >= 10) | 
    (df_per['per_dia'] >= 10)
].reset_index(drop=True)

In [13]:
filtered_df.head()

,N,dim,alpha_a,alpha_g,per_short,std_short,per_ass,std_ass,per_dia,std_dia
0,320000.0,2.0,5.0,2.0,1.009802,0.533346,19.768038,0.002123,1.245705,1.676163
1,160000.0,2.0,5.0,2.0,1.273590,0.859711,80.525082,0.004135,1.828252,3.123525
2,20000.0,2.0,6.0,2.0,0.203844,0.712821,18.238644,0.008148,0.313894,2.675200
3,320000.0,3.0,8.0,2.0,0.605613,0.584853,27.449370,0.002738,1.119827,2.697288
4,80000.0,3.0,8.0,2.0,0.370361,0.783319,913.284603,0.004736,0.518278,2.708013


In [ ]:
# Just assortativity with % standard error >= 10%
filtered_df[filtered_df["per_short"]>=10],filtered_df[filtered_df["per_dia"]>=10], filtered_df[filtered_df["per_ass"]>=10]

(Empty DataFrame
 Columns: [N, dim, alpha_a, alpha_g, per_short, std_short, per_ass, std_ass, per_dia, std_dia]
 Index: [],
 Empty DataFrame
 Columns: [N, dim, alpha_a, alpha_g, per_short, std_short, per_ass, std_ass, per_dia, std_dia]
 Index: [],
            N  dim  alpha_a  alpha_g  per_short  std_short      per_ass  \
 0   320000.0  2.0      5.0      2.0   1.009802   0.533346    19.768038   
 1   160000.0  2.0      5.0      2.0   1.273590   0.859711    80.525082   
 2    20000.0  2.0      6.0      2.0   0.203844   0.712821    18.238644   
 3   320000.0  3.0      8.0      2.0   0.605613   0.584853    27.449370   
 4    80000.0  3.0      8.0      2.0   0.370361   0.783319   913.284603   
 5   160000.0  3.0      8.0      2.0   0.842780   0.513970    39.651905   
 6     5000.0  2.0      8.0      2.0   0.150114   0.700082    17.066217   
 7    40000.0  3.0      9.0      2.0   0.221252   0.639382    17.198788   
 8   160000.0  3.0      9.0      2.0   0.829488   0.551790    13.588838   
 9

In [9]:
N_filt = [i for i in filtered_df["N"]]
dim_filt = [i for i in filtered_df["dim"]]
alpha_a_filt = [i for i in filtered_df["alpha_a"]]
alpha_g_filt = [i for i in filtered_df["alpha_g"]]

comb = {"N":N_filt, "dim": dim_filt, "alpha_a":alpha_a_filt, "alpha_g":alpha_g_filt}
df_remover = pd.DataFrame(comb)

df_non_10 = df.merge(df_remover, on=['N', 'dim', 'alpha_a', 'alpha_g'], how='left', indicator=True)
df_non_10 = df_non_10[df_non_10['_merge'] == 'left_only'].drop(columns=['_merge'])

In [10]:
filtered_df

,N,dim,alpha_a,alpha_g,per_short,std_short,per_ass,std_ass,per_dia,std_dia
0,320000.0,2.0,5.0,2.0,1.009802,0.533346,19.768038,0.002123,1.245705,1.676163
1,160000.0,2.0,5.0,2.0,1.273590,0.859711,80.525082,0.004135,1.828252,3.123525
2,20000.0,2.0,6.0,2.0,0.203844,0.712821,18.238644,0.008148,0.313894,2.675200
3,320000.0,3.0,8.0,2.0,0.605613,0.584853,27.449370,0.002738,1.119827,2.697288
4,80000.0,3.0,8.0,2.0,0.370361,0.783319,913.284603,0.004736,0.518278,2.708013
5,160000.0,3.0,8.0,2.0,0.842780,0.513970,39.651905,0.004045,1.446716,2.211540
6,5000.0,2.0,8.0,2.0,0.150114,0.700082,17.066217,0.013941,0.221468,2.466709
7,40000.0,3.0,9.0,2.0,0.221252,0.639382,17.198788,0.006268,0.331079,2.329253
8,160000.0,3.0,9.0,2.0,0.829488,0.551790,13.588838,0.002820,1.692007,2.774341
9,80000.0,3.0,9.0,2.0,0.338416,0.744308,10.390883,0.005285,0.474524,2.552405


In [11]:
df_non_10

,N,dim,alpha_a,alpha_g,N_samples,short_mean,short_err,short_std,diameter_mean,diameter_err,diameter_std,ass_coeff_mean,ass_coeff_err,ass_coeff_std
0,5000,1,8.0,3.0,51,15.264582,0.141807,1.012706,36.254902,0.510166,3.643312,0.040476,0.001568,0.011196
1,5000,1,4.0,4.0,51,14.834792,0.175089,1.250386,35.686275,0.561853,4.012432,0.015934,0.001676,0.011971
2,5000,1,1.0,9.0,51,11.606915,0.150811,1.077005,28.274510,0.293830,2.098365,-0.095851,0.003681,0.026290
3,5000,1,2.0,14.0,9787,14.726629,0.016291,1.611699,34.694391,0.040366,3.993390,-0.039712,0.000165,0.016287
4,5000,1,0.0,2.0,1052,8.583539,0.015994,0.518760,22.595057,0.055319,1.794243,-0.075066,0.000479,0.015540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717,320000,4,2.0,8.0,8,12.417075,0.167896,0.474883,35.625000,0.460493,1.302470,-0.018685,0.001481,0.004190
2718,320000,4,3.0,2.0,7,12.303786,0.347128,0.918413,34.714286,1.084837,2.870208,-0.032692,0.005251,0.013894
2719,320000,4,7.0,2.0,23,17.931000,0.177517,0.851343,46.478261,0.517937,2.483941,-0.012308,0.000837,0.004014
2720,320000,4,2.0,1.0,16,21.231475,0.783258,3.133030,51.625000,1.154249,4.616998,-0.059372,0.007654,0.030617


In [27]:
mean_std_N_for_dim = {"N":[], "dim":[], "ass_coeff_std_mean":[]}
for n in N:
    for dim in dimensions:
        df_N_dim = df_non_10[(df_non_10["N"]==n) & (df_non_10["dim"]==dim)]
        
        mean_std_N_for_dim["N"].append(n)
        mean_std_N_for_dim["dim"].append(dim)
        mean_std_N_for_dim["ass_coeff_std_mean"].append(df_N_dim["ass_coeff_std"].mean())

df_std_N_dim = pd.DataFrame(data=mean_std_N_for_dim)

In [28]:
df_std_N_dim

,N,dim,ass_coeff_std_mean
0,5000,1,0.013431
1,5000,2,0.015631
2,5000,3,0.016520
3,5000,4,0.017091
4,10000,1,0.070083
5,10000,2,0.012583
6,10000,3,0.013180
7,10000,4,0.013825
8,20000,1,0.007692
9,20000,2,0.009598


In [26]:
mean_std_N_for_dim

{'N': [5000,
  5000,
  5000,
  5000,
  10000,
  10000,
  10000,
  10000,
  20000,
  20000,
  20000,
  20000,
  40000,
  40000,
  40000,
  40000,
  80000,
  80000,
  80000,
  80000,
  160000,
  160000,
  160000,
  160000,
  320000,
  320000,
  320000,
  320000],
 'dim': [1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4],
 'ass_coeff_std_mean': [np.float64(0.013431088438542209),
  np.float64(0.015630982008337745),
  np.float64(0.016520446247152865),
  np.float64(0.017090643204956957),
  np.float64(0.07008310902682545),
  np.float64(0.012583474959363117),
  np.float64(0.013180114314914523),
  np.float64(0.013824676942091873),
  np.float64(0.007691930107764889),
  np.float64(0.00959784462380042),
  np.float64(0.010720382349808388),
  np.float64(0.011304457385145787),
  np.float64(0.00569310152953467),
  np.float64(0.007686883044585463),
  np.float64(0.008664310288512302),
  np.float64(0.0092595809711473